In [1]:
train <- read.csv('./train_cleaned.csv')
test <- read.csv('./test_cleaned.csv')
test$label <- 0

show_level_diff <- function(df1, df2){
    for (i in colnames(df2)){
    n1 = levels(df2[, i])
    n2 = levels(df1[, i])
    if (length(n1) != length(n2) | sum(n1 != n2)){
        print(i)
        print(levels(df1[, i]))
        print(levels(df2[, i]))
        }
    }
}

In [2]:
train_selected <- train
test_selected <- test
# train_selected$chromosome_interact <- NULL


test_selected$protein <- test$protein
test_selected$X82 = NULL
# test_selected$chromosome_interact <- NULL

test_selected$essential[test_selected$essential == 'Ambiguous-Non-Essential'] = 'Non-Essential'
test_selected$essential_interact[test_selected$essential_interact == 'Ambiguous-Essential'] = 'Essential'

character_vars_s <- lapply(train_selected, class) == "character"
train_selected[, character_vars_s] <- lapply(train_selected[, character_vars_s], factor)
character_vars_t <- lapply(test_selected, class) == "character"
test_selected[, character_vars_t] <- lapply(test_selected[, character_vars_t], factor)

for (i in colnames(test_selected)){
    if (i == 'protein'){
        next
    }
    n1 = length(levels(test_selected[, i]))
    n2 = length(levels(test_selected[, i]))
    if (n1 == 1 || n1 != n2){
        test_selected[, i] = NULL
        train_selected[, i] = NULL
    } 
}

In [3]:
library(randomForest)
rf.clf.label <- randomForest(label ~ . - protein, data=train_selected)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.



In [22]:
var.imp <- importance(rf.clf.label)
var.imp <- var.imp[order(var.imp, decreasing = TRUE),, drop=FALSE]
var.imp <- var.imp[var.imp > 10, , drop=FALSE]
train_selected <- train_selected[, names(train_selected) %in% c(rownames(var.imp), 'label', 'protein')]
test_selected <- test_selected[, names(test_selected) %in% c(rownames(var.imp), 'protein')]
test_selected$X51 <- NULL
train_selected$X51 <- NULL

character_vars_s <- lapply(train_selected, class) == "character"
train_selected[, character_vars_s] <- lapply(train_selected[, character_vars_s], factor)
character_vars_t <- lapply(test_selected, class) == "character"
test_selected[, character_vars_t] <- lapply(test_selected[, character_vars_t], factor)
train_selected$label <- factor(train_selected$label)

train_selected$chromosome <- factor(train_selected$chromosome)
test_selected$chromosome <- factor(test_selected$chromosome)

train_size <- floor(0.7 * nrow(train_selected))
train_ind <- sample(seq_len(nrow(train_selected)), size = train_size, replace = FALSE)
valid_ind <- seq_len(nrow(train_selected))[-train_ind]
train_ind <- sample(seq_len(length(train_ind)), size = floor(2 * train_size), replace = TRUE)
# train_selected_b <- train_selected[train_ind, ]
train_selected_df <- train_selected[train_ind, ]
valid_selected_df <- train_selected[valid_ind, ]

# train_ind_0 <- sample(which(train_selected_b$label == 0), size = 1000, replace= TRUE)
# train_selected_df <- train_selected_b[train_ind_0, ]
# train_ind_1 <- sample(which(train_selected_b$label == 1), size = 600, replace = TRUE)
# train_selected_df <- rbind(train_selected_df, train_selected_b[train_ind_1, ])
# train_ind_2 <- sample(which(train_selected_b$label != 1 & train_selected_b$label != 0), size = 800, replace = TRUE)
# train_selected_df <- rbind(train_selected_df, train_selected_b[train_ind_2, ])


train_selected_df$X82 <- NULL
valid_selected_df$X82 <- NULL
test_selected$X82 <- NULL

train_selected_df$X51 <- NULL
valid_selected_df$X51 <- NULL
test_selected$X51 <- NULL


In [23]:
table(train_selected_df$label) / length(train_selected_df$label)
table(valid_selected_df$label) / length(valid_selected_df$label)


           0            1            2            3            4            5 
0.4320066335 0.2247097844 0.0704809287 0.0572139303 0.0613598673 0.0522388060 
           6            7            8            9           10           11 
0.0505804312 0.0207296849 0.0082918740 0.0165837479 0.0008291874 0.0049751244 
          12           13           14 
0.0000000000 0.0000000000 0.0000000000 


          0           1           2           3           4           5 
0.444015444 0.208494208 0.092664093 0.069498069 0.057915058 0.054054054 
          6           7           8           9          10          11 
0.023166023 0.007722008 0.011583012 0.011583012 0.011583012 0.003861004 
         12          13          14 
0.003861004 0.000000000 0.000000000 

In [31]:
library(xgboost)
labels <- train_selected_df$label 
vl_label <- valid_selected_df$label
new_tr <- model.matrix(~.+0,data = train_selected_df[, !names(train_selected_df) %in% c("protein", "label")]) 
new_vl <- model.matrix(~.+0,data = valid_selected_df[, !names(valid_selected_df) %in% c("protein", "label")])
new_ts <- model.matrix(~.+0,data = test_selected[,!names(test_selected) %in% c("protein")])
labels <- as.numeric(labels) - 1
vl_label <- as.numeric(vl_label) - 1
dtrain <- xgb.DMatrix(data = new_tr,label = labels) 
dvalid <- xgb.DMatrix(data = new_vl,label= vl_label)
dtest <- xgb.DMatrix(data = new_ts)
params = list(
  booster="gbtree",
  eta=0.1,
  max_depth=7,
  gamma=1,
  subsample=0.7,
  colsample_bytree=0.8,
  objective="multi:softprob",
  eval_metric="mlogloss",
  num_class=15
)
xgb1 <- xgb.train (params = params, data = dtrain, nrounds = 2000, verbose = 0)

In [32]:
library(caret)
xgb.valid = predict(xgb1, dvalid ,reshape=T)
xgb.valid = as.data.frame(xgb.valid)
colnames(xgb.valid) = levels(train_selected$label)
xgb.valid$prediction = apply(xgb.valid,1,function(x) colnames(xgb.valid)[which.max(x)])
xgb.valid$prediction = as.numeric(xgb.valid$prediction)
accu <- xgb.valid$prediction == valid_selected_df$label
sum(accu) / length(accu)
table(xgb.valid$prediction) 
predicted <- xgb.valid$prediction
confusionMatrix(factor(predicted), factor(valid_selected_df$label))
# accuracy_per_label <- function(label){
#     accu_l_0 <- valid_selected_df$label == label
#     accu_l_0 <- predicted[accu_l_0] == label
#     print(sum(accu_l_0) / length(accu_l_0))
# }
# for (i in levels(valid_selected_df$label)){
#     accuracy_per_label(i)
# }   

[1] 0.8262548


  0   1   2   3   4   5   6   7   8   9 
118  68  18  17  13  15   5   2   1   2 

Warning message in levels(reference) != levels(data):
“longer object length is not a multiple of shorter object length”
Warning message in confusionMatrix.default(factor(predicted), factor(valid_selected_df$label)):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Confusion Matrix and Statistics

          Reference
Prediction   0   1   2   3   4   5   6   7   8   9  10  11  12
        0  107   3   3   2   0   1   0   0   0   0   1   0   1
        1    3  48   6   0   3   2   1   0   1   2   1   1   0
        2    3   0  14   0   0   0   1   0   0   0   0   0   0
        3    1   1   0  15   0   0   0   0   0   0   0   0   0
        4    0   1   0   1  11   0   0   0   0   0   0   0   0
        5    1   1   1   0   1  11   0   0   0   0   0   0   0
        6    0   0   0   0   0   0   4   0   1   0   0   0   0
        7    0   0   0   0   0   0   0   2   0   0   0   0   0
        8    0   0   0   0   0   0   0   0   1   0   0   0   0
        9    0   0   0   0   0   0   0   0   0   1   1   0   0
        10   0   0   0   0   0   0   0   0   0   0   0   0   0
        11   0   0   0   0   0   0   0   0   0   0   0   0   0
        12   0   0   0   0   0   0   0   0   0   0   0   0   0

Overall Statistics
                                          
  

In [8]:
xgb.pred = predict(xgb1, dtest,reshape=T)
xgb.pred = as.data.frame(xgb.pred)
colnames(xgb.pred) = levels(train_selected$label)
xgb.pred$prediction = apply(xgb.pred,1,function(x) colnames(xgb.pred)[which.max(x)])
xgb.pred$prediction = as.numeric(xgb.pred$prediction)
table(xgb.pred$prediction) / length(xgb.pred$prediction)                            


          0           1           2           3           4           5 
0.456692913 0.270341207 0.097112861 0.062992126 0.026246719 0.044619423 
          6           7           8           9 
0.028871391 0.002624672 0.007874016 0.002624672 

In [39]:
test_selected$label <- xgb.pred$prediction

In [40]:
result_agg <- data.frame(Key=test_selected$protein, Label=test_selected$label)
write.csv(result_agg,'./result/result_last.csv', row.names = FALSE, quote=FALSE)

In [ ]:
# params = list(
#   booster="gbtree",
#   eta=0.01,
#   max_depth=5,
#   gamma=3,
#   subsample=0.75,
#   colsample_bytree=1,
#   objective="multi:softprob",
#   eval_metric="mlogloss",
#   num_class=15
# )
# xgbcv <- xgb.cv(params = params, nrounds = 1000, data = dtrain, nfold = 10, early_stopping_rounds = 20)



# searchGridSubCol <- expand.grid(subsample = c(0.7, 0.8), 
#                                 colsample_bytree = c(0.5, 0.8, 1),
#                                 max_depth = c(3, 4, 5),
#                                 min_child = seq(1), 
#                                 eta = c(0.1, 0.01, 0.001)
# )

# ntrees <- 1000   

# system.time(
# rmseErrorsHyperparameters <- apply(searchGridSubCol, 1, function(parameterList){
  
#   #Extract Parameters to test
#   currentSubsampleRate <- parameterList[["subsample"]]
#   currentColsampleRate <- parameterList[["colsample_bytree"]]
#   currentDepth <- parameterList[["max_depth"]]
#   currentEta <- parameterList[["eta"]]
#   currentMinChild <- parameterList[["min_child"]]
#   xgboostModelCV <- xgb.cv(data =  dtrain, nrounds = ntrees, nfold = 5, showsd = TRUE,  verbose = TRUE,
#                      eval_metric="mlogloss", "max.depth" = currentDepth, "eta" = currentEta,                               
#                      "subsample" = currentSubsampleRate, "colsample_bytree" = currentColsampleRate
#                       , print_every_n = 100, "min_child_weight" = currentMinChild, booster = "gbtree",
#                      early_stopping_rounds = 10, num_class = 15)
#   xvalidationScores <- min(xgboostModelCV$evaluation_log[, 'test_mlogloss_mean'])
#   output <- return(c(xvalidationScores, currentSubsampleRate, currentColsampleRate, currentDepth, currentEta, currentMinChild))}))